In [404]:
import pandas as pd
import numpy as np

import datetime as dt

from scipy.stats import ttest_ind, ttest_rel, ttest_1samp
from scipy.stats import f_oneway, pearsonr, chisquare, chi2_contingency
from scipy.stats import shapiro, levene, bartlett

In [405]:
# 데이터 불러오기
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e6_p1_1.csv')

df.head()

,구급보고서번호,소방서명,환자증상구분명1,신고일자,신고시각,출동일자,출동시각,환자직업명,사고발생장소
0,20211122103M01526,중랑소방서,심계항진,20210330,104800,20210330,105042,무직,집
1,20211111105M00771,노원소방서,복통,20210412,225600,20210412,225827,기타,집
2,20211115104M00864,서초소방서,심계항진,20210303,194100,20210303,194500,직장인,상업시설
3,20211113107M01869,마포소방서,고열,20211225,235400,20211226,331,유아,집
4,20211118102M05820,강동소방서,고열,20211019,12100,20211019,12322,무직,집


In [406]:
df['신고일자'] = df['신고일자'].astype('str')
df['신고시각'] = df['신고시각'].astype('str')

df['신고시각'] = df['신고시각'].map(lambda x:str(x).zfill(6))

df['신고_DATE'] = df['신고일자'] + df['신고시각']

print(df['신고_DATE'].head(10))

0    20210330104800
1    20210412225600
2    20210303194100
3    20211225235400
4    20211019012100
5    20210810122800
6    20211024091500
7    20210315235900
8    20210228091500
9    20210729124900
Name: 신고_DATE, dtype: object


In [407]:
df['출동일자'] = df['출동일자'].astype('str')
df['출동시각'] = df['출동시각'].astype('str')

df['출동시각'] = df['출동시각'].map(lambda x:str(x).zfill(6))

df['출동_DATE'] = df['출동일자'] + df['출동시각']

print(df['출동_DATE'].head(10))

0    20210330105042
1    20210412225827
2    20210303194500
3    20211226000331
4    20211019012322
5    20210810122900
6    20211024091725
7    20210316000239
8    20210228091900
9    20210729125146
Name: 출동_DATE, dtype: object


In [408]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2762 entries, 0 to 2761
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   구급보고서번호   2762 non-null   object
 1   소방서명      2762 non-null   object
 2   환자증상구분명1  2762 non-null   object
 3   신고일자      2762 non-null   object
 4   신고시각      2762 non-null   object
 5   출동일자      2762 non-null   object
 6   출동시각      2762 non-null   object
 7   환자직업명     2762 non-null   object
 8   사고발생장소    2762 non-null   object
 9   신고_DATE   2762 non-null   object
 10  출동_DATE   2762 non-null   object
dtypes: object(11)
memory usage: 237.5+ KB


##### 문제 1-1
- 각 구급보고서별 출동시각과 신고시각의 차이 -> '소요시간' 컬럼으로 추가 (초 단위)
- 소방서별 소요시간의 평균을 오름차순으로 정렬했을 때
- 3번째로 작은 소요시간 값과 소방서명을 출력하라

In [409]:
df['신고_DATE'] = pd.to_datetime(df['신고_DATE'])
df['출동_DATE'] = pd.to_datetime(df['출동_DATE'])

df['소요시각'] = (df['출동_DATE']-df['신고_DATE']).dt.total_seconds()

In [410]:
res = df.groupby(['소방서명'])['소요시각'].mean()
result = res.sort_values().reset_index()

print(result.iloc[2].values)

['종로소방서' 175.5]


In [411]:
# df['소요시각'] = (
#                 pd.to_datetime(
#                     df['출동일자'].astype('str') + df['출동시각'].astype('str').str.zfill(6)
#                 )

#                 - 
    
#                 pd.to_datetime(
#                     df['신고일자'].astype('str') + df['신고시각'].astype('str').str.zfill(6)
#                 )
#                 ).dt.total_seconds()


result = df.groupby(['소방서명'])['소요시각'].mean().sort_values().reset_index().iloc[2].values
print(result)

['종로소방서' 175.5]


In [412]:
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e6_p1_2.csv')

df.head()

,학교명,시도,학교세부유형,일반학급_학생수_계,교원수_총계_계
0,서울대학교사범대학부설고등학교,서울,일반고등학교,689,73
1,가락고등학교,서울,일반고등학교,765,70
2,개포고등학교,서울,일반고등학교,684,69
3,경기고등학교,서울,일반고등학교,1047,96
4,경동고등학교,서울,일반고등학교,529,58


##### 문제1-2

- 학교세부유형이 일반중학교인 학교들 중
- 일반중학교 숫자가 2번째로 많은 시도의 일반중학교 데이터만 필터
- 해당 시도의 교원 한명 당 맡은 학생수가 가장 많은 학교 찾기
- 해당 학교의 교원수 출력

In [413]:
cond1 = df['학교세부유형'] == '일반중학교'

print(df['학교세부유형'].value_counts())
print(df.info())

일반중학교     3260
일반고등학교    1700
Name: 학교세부유형, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4960 entries, 0 to 4959
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   학교명         4960 non-null   object
 1   시도          4960 non-null   object
 2   학교세부유형      4960 non-null   object
 3   일반학급_학생수_계  4960 non-null   int64 
 4   교원수_총계_계    4960 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 193.9+ KB
None


In [414]:
df1 = df[cond1]

df1.head()

,학교명,시도,학교세부유형,일반학급_학생수_계,교원수_총계_계
230,동원중학교,서울,일반중학교,379,36
231,면목중학교,서울,일반중학교,462,42
232,봉화중학교,서울,일반중학교,256,31
233,성일중학교,서울,일반중학교,313,30
234,신현중학교,서울,일반중학교,683,52


In [415]:
region = df1.groupby('시도').count().sort_values('학교세부유형', ascending=False).reset_index().iloc[1,0]

print(region)

서울


In [416]:
df2 = df.copy()

cond2 = df2['시도'] == region

df2['맡은학생수'] = df2['일반학급_학생수_계']/df2['교원수_총계_계']

df3 = df2[cond2].sort_values('맡은학생수', ascending=False)
teacher_num = df2[cond2].sort_values('맡은학생수', ascending=False).reset_index().loc[0,'교원수_총계_계']

# print(df3)
print(round(teacher_num))


33


##### 문제1-3

- 5대 범죄(절도,사기,배임,방화,폭행)의 월별 총 발생건수를 총범죄수로 표현
- 18, 19년의 각각 분기별 총범죄수의 월평균 값을 구했을 때
- 최대값을 가지는 년도와 분기를 구하고
- 해당 분기의 최댓값의 사기가 발생한 월의 사기 발생건수 출력

* 1분기(1,2,3월) / 2분기(4,5,6월) / 3분기 (7,8,9월) / 4분기 (10,11,12월)

In [417]:
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e6_p1_3.csv')

df.head()

,범죄분류,절도,사기,배임,방화,폭행
0,2018년_1월,13280,25318,656,143,12186
1,2018년_2월,12504,19159,403,135,12087
2,2018년_3월,14733,24509,471,148,13841
3,2018년_4월,14781,23147,414,135,13973
4,2018년_5월,15988,23584,451,127,14117


In [418]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   범죄분류    24 non-null     object
 1   절도      24 non-null     int64 
 2   사기      24 non-null     int64 
 3   배임      24 non-null     int64 
 4   방화      24 non-null     int64 
 5   폭행      24 non-null     int64 
dtypes: int64(5), object(1)
memory usage: 1.2+ KB


In [419]:
data_list = df['범죄분류'].str.split('_')

print(data_list)

0      [2018년, 1월]
1      [2018년, 2월]
2      [2018년, 3월]
3      [2018년, 4월]
4      [2018년, 5월]
5      [2018년, 6월]
6      [2018년, 7월]
7      [2018년, 8월]
8      [2018년, 9월]
9     [2018년, 10월]
10    [2018년, 11월]
11    [2018년, 12월]
12     [2019년, 1월]
13     [2019년, 2월]
14     [2019년, 3월]
15     [2019년, 4월]
16     [2019년, 5월]
17     [2019년, 6월]
18     [2019년, 7월]
19     [2019년, 8월]
20     [2019년, 9월]
21    [2019년, 10월]
22    [2019년, 11월]
23    [2019년, 12월]
Name: 범죄분류, dtype: object


In [420]:
df1 = df.copy()

df2 = pd.DataFrame({
    '년도' : data_list.str.get(0),
    '월' : data_list.str.get(1)
})

df3 = pd.concat([df1,df2],axis=1)

df3.head()

,범죄분류,절도,사기,배임,방화,폭행,년도,월
0,2018년_1월,13280,25318,656,143,12186,2018년,1월
1,2018년_2월,12504,19159,403,135,12087,2018년,2월
2,2018년_3월,14733,24509,471,148,13841,2018년,3월
3,2018년_4월,14781,23147,414,135,13973,2018년,4월
4,2018년_5월,15988,23584,451,127,14117,2018년,5월


In [421]:
df3['총범죄수'] = df3['절도'] + df3['사기'] + df3['배임'] + df3['방화'] + df3['폭행']

df3.head()

,범죄분류,절도,사기,배임,방화,폭행,년도,월,총범죄수
0,2018년_1월,13280,25318,656,143,12186,2018년,1월,51583
1,2018년_2월,12504,19159,403,135,12087,2018년,2월,44288
2,2018년_3월,14733,24509,471,148,13841,2018년,3월,53702
3,2018년_4월,14781,23147,414,135,13973,2018년,4월,52450
4,2018년_5월,15988,23584,451,127,14117,2018년,5월,54267


In [422]:
df3['분기'] = 'x분기'

for i in range(4) :
    month_to_check = [f'{i*3+1}월', f'{i*3+2}월', f'{i*3+3}월']
    # print(month_to_check)
    df3['분기'] = np.where(df3['월'].isin(month_to_check), f'{i+1}분기', df3['분기'])

# for i in range(0, 12, 3):  # 0부터 11까지 3씩 증가하는 범위
#     months_to_check = [f'{j+1}월' for j in range(i, i+3)]
#     # 해당 월들이 있는지 확인하고, 있으면 분기를 지정합니다.
#     df3['분기'] = np.where(df3['월'].isin(months_to_check), f'{(i//3) + 1}분기', df3['분기'])


df3

,범죄분류,절도,사기,배임,방화,폭행,년도,월,총범죄수,분기
0,2018년_1월,13280,25318,656,143,12186,2018년,1월,51583,1분기
1,2018년_2월,12504,19159,403,135,12087,2018년,2월,44288,1분기
2,2018년_3월,14733,24509,471,148,13841,2018년,3월,53702,1분기
3,2018년_4월,14781,23147,414,135,13973,2018년,4월,52450,2분기
4,2018년_5월,15988,23584,451,127,14117,2018년,5월,54267,2분기
5,2018년_6월,15744,23001,453,110,14362,2018년,6월,53670,2분기
6,2018년_7월,16140,24990,632,98,14339,2018년,7월,56199,3분기
7,2018년_8월,16327,24463,434,102,14824,2018년,8월,56150,3분기
8,2018년_9월,16091,22073,486,130,15256,2018년,9월,54036,3분기
9,2018년_10월,15122,21769,531,107,13286,2018년,10월,50815,4분기


In [423]:
gr = df3.groupby(['년도','분기']).mean(numeric_only=True).sort_values('총범죄수',ascending=False).reset_index().loc[0,['년도','분기']]

print(gr['년도'])
print(gr['분기'])

2019년
2분기


In [424]:
cond3 = (df3['분기'] == gr['분기'])
cond5 = (df3['년도'] == gr['년도'])

df4 = df3[cond3 & cond5].sort_values('사기', ascending=False)

print(df4.head())

max_ev = df4['월'].iloc[0]

print(max_ev)

        범죄분류     절도     사기   배임   방화     폭행     년도   월   총범죄수   분기
16  2019년_5월  17333  27766  509  131  14572  2019년  5월  60311  2분기
15  2019년_4월  15305  27125  478  138  13322  2019년  4월  56368  2분기
17  2019년_6월  16937  25822  491  101  15285  2019년  6월  58636  2분기
5월


In [425]:
cond4 = (df3['월'] == max_ev)

res = df3[cond4]['사기'].max()

print(res)

27766
